# Introduction à PyTorch : Dernier cours sur la construction des modèles

----

Cette partie est la dernière partie de cette série de cours et traite de la création de modèles. Elle constitue alors un approfondissement car c'est là que réside les variantes dans l'utilisation de PyTorch...

Ce qui va alors s'avérer important est de comprendre comment sont implémentées les couches de notre réseau avec PyTorch, comment les incorporer dans notre modèle efficacement et finalement comment les appliquer sur nos données correctement.

Ainsi nous allons voir le cas d'un réseau simple de classification avec l'utilisation de fonction d'activation (non utilisée jusqu'à maintenant), le cas d'un CNN pour de la classification d'images simples et pour finir, un RNN simple et pour finir une application sur une architecture plus compliquée sera mise dans un fichier à part qui constituera alors la dernière étude de cas pour clôturer ce cours (on verra un U-net, l'architecture étant très visuelle mais assez complexe).

----

## Préparation des données:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
# from multiprocessing import freeze_support
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # GPU or CPU


### Cifar10 pour le CNN

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # ça permet de transformer les données en tenseurs et de les normaliser quand on les récupère des datasets

batch_size_cifar = 4

trainset_cifar10 = torchvision.datasets.CIFAR10(root='./PyTorch_ver/data_pytorch', train=True,
                                        download=True, transform=transform) # On charge le dataset avec les transformations

trainloader_cifar10 = torch.utils.data.DataLoader(trainset_cifar10, batch_size=batch_size_cifar,
                                        shuffle=True, num_workers=2) # On charge le dataset dans un loader qui nous permettra de le parcourir avec une boucle for

testset_cifar10 = torchvision.datasets.CIFAR10(root='./PyTorch_ver/data_pytorch', train=False,
                                    download=True, transform=transform)
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10, batch_size=batch_size_cifar,
                                        shuffle=False, num_workers=2) # Pareil qu'avec le trainloader

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### MNIST pour le classifieur

In [ ]:
# Hyper-parameters
input_size = 784 # Les images sont de taille 28*28 = 784
hidden_size = 500
num_classes = 10
num_epochs = 11
batch_size_mnist = 100
learning_rate = 0.001

#MNIST Dataset :
train_dataset_mnist = torchvision.datasets.MNIST(root='./PyTorch_ver/data_pytorch', train=True, transform=transforms.ToTensor(), download=True)

test_dataset_mnist = torchvision.datasets.MNIST(root='./PyTorch_ver/data_pytorch', train=False, transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader_mnist = torch.utils.data.DataLoader(dataset=train_dataset_mnist, batch_size=batch_size_mnist, shuffle=True)

test_loader_mnist = torch.utils.data.DataLoader(dataset=test_dataset_mnist, batch_size=batch_size_mnist, shuffle=True)


----

## Définition du modèle:

### CNN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # (input_channels, output_channels, kernel_size)
        self.pool = nn.MaxPool2d(2, 2) # (kernel_size, stride)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # le 16*5*5 correspond au nombre de filtres de la couche de convolution précédente fois le kernel size (5x5)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        # On a donc définie un total de 6 couches : 2 couches de convolution, 3 couches linéaires et une couche de pooling
        # On doit impérativement définir ici le nombre de layer correspondant à l'architecture du réseau. La raison pour cela est que
        # chaque couches est définie par un module nn.Module et contient ainsi des paramètres qui seront optimisés par le réseau de neurones.

    def forward(self, x):
        
        x = self.conv1(x) # On applique la première couche de convolution
        x = F.relu(x) # On applique la fonction d'activation
        x = self.pool(x) # On applique la couche de pooling
        
        x = self.pool(F.relu(self.conv2(x))) # On effectue les 3 opérations précédentes en une seule ligne
        
        x = torch.flatten(x, 1) # On applati les tenseurs pour les passer dans les couches linéaires (sauf la première dimension qui est la taille du batch)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        # On définit ici la fonction forward qui permet de définir le passage des données dans le réseau de neurones.
        # On peut littéralement voir ça comme une fonction mathématique qui prend en entrée un tenseur et qui renvoie un tenseur et rien de plus.
        
        return x

### Classifieur

In [ ]:
class NeuralNetwork(nn.Module):
    
    def __init__(self, input_size = input_size, hidden_size = hidden_size, num_classes= num_classes):
        
        super(NeuralNetwork, self).__init__()
        
        self.dense = nn.Linear(input_size, hidden_size)
        
        self.relu = nn.ReLU() # Autre manière d'utiliser la fonction d'activation
        
        self.dense_1 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        
        out = self.dense(x)
        
        out = self.relu(out)
        
        out = self.dense_1(out)
        
        return out

----

## Définition de la boucle d'entraînement:

### CNN

In [ ]:
net = Net()
net.to(device) # On envoie le modèle sur le GPU si on en a un pour que tout l'entrainement se fasse sur le GPU

criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(4):  # Boucle sur un nombre d'epoch de 4

        running_loss = 0.0
        for i, data in enumerate(trainloader_cifar10, 0): # On parcourt le trainloader donc tout le dataset
            
            inputs, labels = data 

            # Ici on met les gradients à zéro pour ne pas qu'ils s'accumulent et faussent tout
            optimizer_cnn.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion_cnn(outputs, labels)
            loss.backward()
            optimizer_cnn.step() # MAJ des poids

            # Partie affichage des résultats
            running_loss += loss.item() # On ajoute la loss à la running_loss (somme de la loss sur tout les mini-batchs)
            if i % 2000 == 1999:    # On print tout les 2000 mini-batchs
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

print('Finished Training')

### Notes importantes:

- En ce qui concerne la boucle on peut aussi la personnaliser, l'encapsuler dans une fonction etc... Ici elle est simplement à la suite du code (cas basique).

- La boucle commence par : 
```python 
    for i, data in enumerate(trainloader_cifar10, 0): 
```
Or qu'est-ce que le enumerate ?? C'est une classe en python qui prend en entrée ce qu'on appelle un générateur et dont le but est de fournir deux éléments dans une boucle for. Le premier élément est notre $i$ qui est un compteur et notre deuxième élément est $data$ qui est un objet issu du générateur. Ici notre générateur c'est notre $trainloader\_cifar10$ et quand on l'a définie on a utilisé un objet $DataLoader$. La particularité de cet objet est qu'il permet de créer un générateur donnant comme objet un tuple (x, y) avec x notre input de notre model et y notre label

Ainsi $i$ va compter les éléments présents dans un batch et $data$ est ainsi composé des inputs (donc une image) et de son label (donc une classe) 


### Classifieur

In [ ]:
def training(model = NeuralNetwork(), save = False, num_epochs = num_epochs):
    
    # loss and optimizer :
    loss = nn.CrossEntropyLoss() # computes softmax and then the cross entropy so we don't need activation function at the output layer
    # loss = nn.MSELoss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # We can now train the model :

    n_steps = len(train_loader_mnist)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader_mnist):
            
            images = images.reshape(-1, 28*28).to(device) # Ici on fait un élément de préprocessing pour avoir les images sous la forme d'un vecteur de taille 784
                                                          # et on met sur le GPU si on en a un
                                                          
            labels = labels.to(device) # On met les labels sur le GPU si on en a un
            
            # Forward + Backward + Optimize : 
            
            outputs  = model.forward(images) # Même chose que model(images) mais on utilise la fonction forward pour être plus explicite ici 
            l = loss(outputs, labels)
            
            l.backward()
            optimizer.step() # On met à jour les paramètres du modèle
            optimizer.zero_grad() # On met les gradients à zéro pour ne pas qu'ils s'accumulent et faussent tout
            
            if (i+1)%100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_steps}], Loss: {l.item():.4f}')
    if save:
        torch.save(model.state_dict(), 'model.pt')

### Notes:

Ici on a donc un classifieur d'image et on a utilisé une fonction pour l'entraînement plutôt qu'une boucle. Cela permet donc diverses variations comme le fait de rajouter une sauvegarde de nos poids par exemple ou de changer le nombre d'epoch facilement.

On pourra aussi noter l'utilisation de la commande :
```py
    torch.save(model.state_dict(), 'model.pt')
```

qui se sert d'un dictionnaire qu'il stock dans un fichier .pt. Ce dictionnaire contient tous les poids du réseau, c'est donc une ligne très importante.

----

## Conclusion :

Voilà on a vu deux applications avec deux réseaux différents avec leur pipeline respectives et quelques idées de variantes. Ces variantes permettent de voir comment encapsuler des parties de la pipelines, modifier les hyper-paramètres à divers endroits ou les variantes dans l'affichage ou encore la manière d'appliquer les couches aux inputs dans le réseau. 